In [1]:
# Test cell for oanda_api_service.py

import pandas as pd
# Make sure the other scripts are in the same directory or your Python path
from oanda_api_service import (
    get_oanda_client, 
    get_instrument_details, 
    get_historical_candles, 
    get_open_trades,
    place_trade
)
from live_trader_config import (
    INSTRUMENTS, 
    CANDLE_GRANULARITY, 
    TRADE_UNITS
)

# --- 1. Test API Client Initialization ---
print("--- Testing API Client Initialization ---")
api_client = get_oanda_client()
if api_client:
    print("✅ OANDA API client initialized successfully.")
else:
    print("❌ Failed to initialize OANDA API client. Check API key and environment in config.")
    # Stop execution if client fails
    assert api_client is not None, "Client initialization failed."

# --- 2. Test Fetching Instrument Details ---
# We'll test with the first instrument in your list
test_instrument = INSTRUMENTS[0] 
print(f"\n--- Testing Instrument Details for {test_instrument} ---")
pip_location, display_precision = get_instrument_details(api_client, test_instrument)
if pip_location is not None and display_precision is not None:
    print(f"✅ Successfully fetched details for {test_instrument}:")
    print(f"   - Pip Location: {pip_location}")
    print(f"   - Display Precision: {display_precision}")
else:
    print(f"❌ Failed to fetch details for {test_instrument}.")

# --- 3. Test Fetching Historical Candles ---
print(f"\n--- Testing Historical Candle Fetch for {test_instrument} ---")
candles_df = get_historical_candles(api_client, test_instrument, count=50, granularity=CANDLE_GRANULARITY)
if not candles_df.empty:
    print(f"✅ Successfully fetched {len(candles_df)} candles.")
    print("Sample of the latest data:")
    display(candles_df.tail(3))
else:
    print(f"❌ Failed to fetch historical candles for {test_instrument}.")

# --- 4. Test Fetching Open Trades ---
print(f"\n--- Testing Open Trades Fetch for {test_instrument} ---")
open_trade = get_open_trades(api_client, test_instrument)
if open_trade is not None:
    print(f"✅ Found an open trade for {test_instrument}:")
    print(open_trade)
else:
    # This is not an error, it just means no trade is open.
    print(f"✅ Successfully checked for trades. No open trade found for {test_instrument}.")

# --- 5. Test Placing a Trade (USE WITH CAUTION) ---
# This places a real order in your practice account.
# UNCOMMENT THE FOLLOWING LINES ONLY IF YOU WANT TO TEST PLACING AN ORDER.

# print(f"\n--- Testing Trade Placement for {test_instrument} ---")
# print("WARNING: This will place a live order in your configured account.")
# # Example: Place a BUY order with a dummy SL and TP
# # In a real scenario, SL/TP would be calculated.
# test_sl_price = "1.07000" 
# test_tp_price = "1.08000"
# trade_response = place_trade(
#     api_client, 
#     instrument=test_instrument, 
#     units=1, # Use 1 unit for testing
#     direction="BUY", 
#     sl_price=test_sl_price, 
#     tp_price=test_tp_price
# )

# if trade_response and ("orderFillTransaction" in trade_response or "orderCreateTransaction" in trade_response):
#     print("✅ Trade placement function executed successfully.")
#     print("Response:", trade_response)
# else:
#     print("❌ Trade placement function failed or was rejected.")
#     if trade_response:
#         print("Response:", trade_response)

print("\n--- All Tests Complete ---")

--- Testing API Client Initialization ---
✅ OANDA API client initialized successfully.

--- Testing Instrument Details for EUR_USD ---
✅ Successfully fetched details for EUR_USD:
   - Pip Location: -4
   - Display Precision: 5

--- Testing Historical Candle Fetch for EUR_USD ---
✅ Successfully fetched 50 candles.
Sample of the latest data:


,open,high,low,close,volume
time,,,,,
2025-06-17 18:45:00+00:00,1.14816,1.14816,1.14750,1.14797,2322
2025-06-17 19:00:00+00:00,1.14796,1.14814,1.14778,1.14798,1627
2025-06-17 19:15:00+00:00,1.14800,1.14804,1.14775,1.14777,464



--- Testing Open Trades Fetch for EUR_USD ---
✅ Successfully checked for trades. No open trade found for EUR_USD.

--- All Tests Complete ---


In [3]:
# Test cell for technical_indicator_service.py

import pandas as pd
import numpy as np
from technical_indicator_service import add_all_technical_indicators
import json
from live_trader_config import MODEL_FEATURES_FILE_PATH

# --- 1. Create a realistic sample DataFrame ---
# This simulates the data we get from the OANDA API (e.g., from get_historical_candles)
print("--- Creating a sample DataFrame for testing ---")
np.random.seed(42)
data = {
    'open': 1.1000 + np.random.randn(100).cumsum() * 0.001,
    'high': 1.1000 + np.random.randn(100).cumsum() * 0.001 + 0.0005,
    'low': 1.1000 + np.random.randn(100).cumsum() * 0.001 - 0.0005,
    'close': 1.1000 + np.random.randn(100).cumsum() * 0.001,
    'volume': np.random.randint(100, 1000, 100)
}
# Create a DatetimeIndex, which is crucial for time-based features
index = pd.to_datetime(pd.date_range(start='2025-06-18', periods=100, freq='15min'))
sample_df = pd.DataFrame(data, index=index)

print("Sample DataFrame created with 100 rows.")
print("Columns:", sample_df.columns.tolist())
print("Index type:", type(sample_df.index))
display(sample_df.head(3))


# --- 2. Run the indicator calculation function ---
print("\n--- Testing add_all_technical_indicators function ---")
df_with_indicators = add_all_technical_indicators(sample_df)

if isinstance(df_with_indicators, pd.DataFrame):
    print("✅ Function returned a pandas DataFrame.")
else:
    print("❌ Function did not return a DataFrame.")
    assert False, "Test failed: incorrect return type."

# --- 3. Verify the output ---
print("\n--- Verifying the results ---")

# Load the list of features the model was trained on
try:
    with open(MODEL_FEATURES_FILE_PATH, 'r') as f:
        model_features = json.load(f)
    print(f"Loaded {len(model_features)} model features from '{MODEL_FEATURES_FILE_PATH}' for comparison.")
except FileNotFoundError:
    print(f"Warning: '{MODEL_FEATURES_FILE_PATH}' not found. Using a predefined list for testing.")
    # Define a fallback list if the JSON is not available
    model_features = [
        'profit_potential', 'break_type', 'bb_percent', 'rsi', 'macd', 
        'macd_signal', 'price_change_1', 'price_change_5', 'volatility', 
        'hour', 'day_of_week', 'volume_ratio'
    ]

# Check if all expected indicator columns were added
# We exclude 'profit_potential' and 'break_type' as they are not added by this service
expected_cols = [col for col in model_features if col not in ['profit_potential', 'break_type']]
missing_cols = [col for col in expected_cols if col not in df_with_indicators.columns]

if not missing_cols:
    print("✅ All expected indicator columns are present in the output DataFrame.")
else:
    print(f"❌ Missing expected columns: {missing_cols}")
    assert False, "Test failed: missing columns."

# Check for NaNs at the beginning (expected) and valid numbers at the end
print("\nChecking for initial NaNs and later valid data...")
# Check the last row for any unexpected NaNs (some might be NaN if window is large, but most should be valid)
last_row_nan_check = df_with_indicators[expected_cols].iloc[-1].isnull().sum()
if last_row_nan_check == 0:
    print("✅ The last row contains valid numbers for all indicators.")
else:
    print(f"⚠️ Warning: The last row contains {last_row_nan_check} NaN values. This might be okay if not enough data was provided.")
    print("NaNs in last row:", df_with_indicators[expected_cols].iloc[-1].isnull())


print("\n--- Displaying results for visual inspection ---")
# Display the first few rows (should have many NaNs)
print("\nHead of the DataFrame (expect NaNs in indicator columns):")
display(df_with_indicators.head())

# Display the last few rows (should be populated with values)
print("\nTail of the DataFrame (expect valid numbers):")
display(df_with_indicators.tail())

print("\n--- All Tests Complete ---")

--- Creating a sample DataFrame for testing ---
Sample DataFrame created with 100 rows.
Columns: ['open', 'high', 'low', 'close', 'volume']
Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,open,high,low,close,volume
2025-06-18 00:00:00,1.100497,1.099085,1.099858,1.099171,198
2025-06-18 00:15:00,1.100358,1.098664,1.100419,1.098611,610
2025-06-18 00:30:00,1.101006,1.098321,1.101502,1.099358,484



--- Testing add_all_technical_indicators function ---
✅ Function returned a pandas DataFrame.

--- Verifying the results ---
✅ All expected indicator columns are present in the output DataFrame.

Checking for initial NaNs and later valid data...
✅ The last row contains valid numbers for all indicators.

--- Displaying results for visual inspection ---

Head of the DataFrame (expect NaNs in indicator columns):


,open,high,low,close,volume,bb_middle,bb_upper,bb_lower,bb_percent,rsi,macd,macd_signal,volume_sma,volume_ratio,price_change_1,price_change_5,volatility,hour,day_of_week
2025-06-18 00:00:00,1.100497,1.099085,1.099858,1.099171,198,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000e+00,198.000000,1.000000,NaN,NaN,NaN,0,2
2025-06-18 00:15:00,1.100358,1.098664,1.100419,1.098611,610,NaN,NaN,NaN,NaN,0.000000,-0.000045,-8.937361e-06,404.000000,1.509901,-0.000510,NaN,0.000396,0,2
2025-06-18 00:30:00,1.101006,1.098321,1.101502,1.099358,484,NaN,NaN,NaN,NaN,57.155495,-0.000020,-1.106497e-05,430.666667,1.123839,0.000680,NaN,0.000389,0,2
2025-06-18 00:45:00,1.102529,1.097519,1.102555,1.099968,504,NaN,NaN,NaN,NaN,70.791119,0.000049,9.504871e-07,449.000000,1.122494,0.000555,NaN,0.000560,0,2
2025-06-18 01:00:00,1.102295,1.097358,1.101178,1.099948,922,NaN,NaN,NaN,NaN,70.027921,0.000101,2.086504e-05,543.600000,1.696100,-0.000019,NaN,0.000570,1,2



Tail of the DataFrame (expect valid numbers):


,open,high,low,close,volume,bb_middle,bb_upper,bb_lower,bb_percent,rsi,macd,macd_signal,volume_sma,volume_ratio,price_change_1,price_change_5,volatility,hour,day_of_week
2025-06-18 23:45:00,1.089288,1.104545,1.103340,1.109920,884,1.108934,1.112089,1.105778,0.656277,50.548541,0.000746,0.000720,506.00,1.747036,-0.000423,0.001546,0.001578,23,2
2025-06-19 00:00:00,1.089584,1.103661,1.104240,1.108207,607,1.108862,1.112015,1.105709,0.396165,52.135432,0.000638,0.000704,509.65,1.191013,-0.001543,-0.000443,0.001577,0,3
2025-06-19 00:15:00,1.089845,1.103815,1.104547,1.109561,951,1.108848,1.111986,1.105710,0.613533,58.763532,0.000654,0.000694,529.90,1.794678,0.001222,0.001971,0.001569,0,3
2025-06-19 00:30:00,1.089850,1.103873,1.105360,1.109446,168,1.108720,1.111501,1.105938,0.630633,61.618448,0.000650,0.000685,517.95,0.324356,-0.000103,0.000213,0.001391,0,3
2025-06-19 00:45:00,1.089615,1.102730,1.105990,1.110684,645,1.108693,1.111384,1.106003,0.869994,64.525882,0.000738,0.000696,532.80,1.210586,0.001116,0.000266,0.001345,0,3



--- All Tests Complete ---


In [2]:
# Test cell for trading_logic_service.py

import pandas as pd
import numpy as np
import joblib
import json

# Import functions to be tested
from trading_logic_service import (
    load_model_and_features, 
    prepare_features_for_prediction, 
    get_trade_decision
)
# Import from other services to create test data
from technical_indicator_service import add_all_technical_indicators
from live_trader_config import MODEL_FEATURES_FILE_PATH

# --- 1. Test Loading the Real Model and Features ---
print("--- 1. Testing loading of the actual model and features ---")
model, model_features = load_model_and_features()
if model and model_features:
    print(f"✅ Successfully loaded model and {len(model_features)} features.")
else:
    print("❌ Failed to load model or features. Ensure files exist and paths in config are correct.")
    assert False, "Test failed: Cannot load model/features."

# --- 2. Create a Mock Model for Predictable Testing ---
# This allows us to force a "trade" or "no trade" prediction.
class MockModel:
    def __init__(self, prediction_value=1):
        self._prediction_value = prediction_value
        print(f"MockModel created. It will always predict: {self._prediction_value}")

    def predict(self, df):
        # The model should return an array-like object
        return np.array([self._prediction_value])

# --- 3. Create Realistic Test Data with Indicators ---
# This creates a base DataFrame with all indicators calculated.
np.random.seed(42)
data = {'close': 1.1000 + np.random.randn(100).cumsum() * 0.0005}
index = pd.to_datetime(pd.date_range(start='2025-06-18', periods=100, freq='15min'))
base_df = pd.DataFrame(data, index=index)
base_df_with_indicators = add_all_technical_indicators(base_df)
print("\n--- 2. Base data with indicators created for testing ---")


# --- 4. Test Scenarios for get_trade_decision ---
print("\n--- 3. Testing get_trade_decision function with scenarios ---")

# Define dummy instrument properties for the test
test_pip_location = -4  # For EUR/USD, etc. (0.0001)
test_display_precision = 5

# SCENARIO A: Upper Break, Model Predicts TRADE (1)
print("\n--- SCENARIO A: Upper Break / Model Predicts TRADE ---")
df_upper_break = base_df_with_indicators.copy()
# Manually force the second-to-last candle to close ABOVE its upper band
df_upper_break.iloc[-2, df_upper_break.columns.get_loc('close')] = df_upper_break.iloc[-2]['bb_upper'] + 0.0001
mock_model_trade = MockModel(prediction_value=1)

direction, sl, tp = get_trade_decision(df_upper_break, mock_model_trade, model_features, test_pip_location, test_display_precision)

if direction == "SELL" and sl and tp:
    print(f"✅ Correctly identified a SELL signal. SL: {sl}, TP: {tp}")
else:
    print(f"❌ Test Failed. Expected 'SELL', got: {direction}")
    assert direction == "SELL"

# SCENARIO B: Lower Break, Model Predicts TRADE (1)
print("\n--- SCENARIO B: Lower Break / Model Predicts TRADE ---")
df_lower_break = base_df_with_indicators.copy()
# Manually force the second-to-last candle to close BELOW its lower band
df_lower_break.iloc[-2, df_lower_break.columns.get_loc('close')] = df_lower_break.iloc[-2]['bb_lower'] - 0.0001

direction, sl, tp = get_trade_decision(df_lower_break, mock_model_trade, model_features, test_pip_location, test_display_precision)

if direction == "BUY" and sl and tp:
    print(f"✅ Correctly identified a BUY signal. SL: {sl}, TP: {tp}")
else:
    print(f"❌ Test Failed. Expected 'BUY', got: {direction}")
    assert direction == "BUY"

# SCENARIO C: No Break
print("\n--- SCENARIO C: No Break ---")
df_no_break = base_df_with_indicators.copy()
# Ensure the close is safely within the bands
df_no_break.iloc[-2, df_no_break.columns.get_loc('close')] = df_no_break.iloc[-2]['bb_middle']

direction, sl, tp = get_trade_decision(df_no_break, mock_model_trade, model_features, test_pip_location, test_display_precision)

if direction is None:
    print("✅ Correctly identified NO signal.")
else:
    print(f"❌ Test Failed. Expected 'None', got: {direction}")
    assert direction is None

# SCENARIO D: Upper Break, Model Predicts NO TRADE (0)
print("\n--- SCENARIO D: Upper Break / Model Predicts NO TRADE ---")
mock_model_no_trade = MockModel(prediction_value=0)
# We can reuse the df_upper_break data from Scenario A
direction, sl, tp = get_trade_decision(df_upper_break, mock_model_no_trade, model_features, test_pip_location, test_display_precision)

if direction is None:
    print("✅ Correctly respected model's 'NO TRADE' prediction.")
else:
    print(f"❌ Test Failed. Expected 'None' due to model prediction, got: {direction}")
    assert direction is None

# --- 5. Test prepare_features_for_prediction function ---
print("\n--- 4. Testing feature preparation function ---")
# Use the last candle from the upper break scenario as input
feature_candle = df_upper_break.iloc[-1]
features_df = prepare_features_for_prediction(
    feature_candle_data=feature_candle,
    is_upper_break=True, # This was the scenario
    is_lower_break=False,
    expected_feature_list=model_features
)

if features_df is not None and not features_df.empty:
    print("✅ Feature preparation function returned a valid DataFrame.")
    if features_df.isnull().values.any():
        print("❌ Test Failed. Prepared features contain NaN values.")
        print(features_df.isnull().sum())
        assert not features_df.isnull().values.any()
    else:
        print("✅ Prepared features contain no NaN values.")
    
    if list(features_df.columns) == model_features:
        print("✅ Feature columns are in the correct order.")
    else:
        print("❌ Test Failed. Feature columns are not in the correct order.")
        assert list(features_df.columns) == model_features
    
    print("Sample of prepared features:")
    display(features_df)
else:
    print("❌ Test Failed. Feature preparation returned None or an empty DataFrame.")
    assert features_df is not None

print("\n--- All Tests Complete ---")

--- 1. Testing loading of the actual model and features ---
✅ Successfully loaded model and 11 features.

--- 2. Base data with indicators created for testing ---

--- 3. Testing get_trade_decision function with scenarios ---

--- SCENARIO A: Upper Break / Model Predicts TRADE ---
MockModel created. It will always predict: 1
Bollinger Band break detected on candle 2025-06-19 00:30:00: Upper=True, Lower=False
Model prediction for features from 2025-06-19 00:45:00: 1
Trade Signal: SELL, SL: 1.09631, TP: 1.09530
✅ Correctly identified a SELL signal. SL: 1.09631, TP: 1.09530

--- SCENARIO B: Lower Break / Model Predicts TRADE ---
Bollinger Band break detected on candle 2025-06-19 00:30:00: Upper=False, Lower=True
Model prediction for features from 2025-06-19 00:45:00: 1
Trade Signal: BUY, SL: 1.09331, TP: 1.09530
✅ Correctly identified a BUY signal. SL: 1.09331, TP: 1.09530

--- SCENARIO C: No Break ---
✅ Correctly identified NO signal.

--- SCENARIO D: Upper Break / Model Predicts NO TRAD

,profit_potential,break_type,bb_percent,rsi,macd,macd_signal,price_change_1,price_change_5,volatility,hour,day_of_week
0,-4.874884,1,0.195182,48.121897,-0.000164,-0.000083,-0.000107,-0.000518,0.0004,0,3



--- All Tests Complete ---


In [5]:
# Test cell for run_live_trade.py using mocking (Corrected)

import pandas as pd
import numpy as np
from unittest.mock import patch, MagicMock

# Import the main function we want to test
from run_live_trade import main
# We need the list of instruments from the config to know what the loop will run on
from live_trader_config import INSTRUMENTS

print("--- Setting up test environment for run_live_trade.py ---")

# --- Create a sample DataFrame that our mock functions will return ---
np.random.seed(42)
data = {'close': 1.1000 + np.random.randn(100).cumsum() * 0.0005}
index = pd.to_datetime(pd.date_range(start='2025-06-18', periods=100, freq='15min'))
sample_candles_df = pd.DataFrame(data, index=index)
from technical_indicator_service import add_all_technical_indicators
sample_df_with_indicators = add_all_technical_indicators(sample_candles_df)


# --- SCENARIO 1: A trade signal is found and NO open trade exists ---
print("\n--- SCENARIO 1: Testing a successful trade signal ---")

with patch('run_live_trade.get_oanda_client') as mock_get_client, \
     patch('run_live_trade.load_model_and_features') as mock_load_model, \
     patch('run_live_trade.get_instrument_details') as mock_get_details, \
     patch('run_live_trade.get_historical_candles') as mock_get_candles, \
     patch('run_live_trade.get_trade_decision') as mock_get_decision, \
     patch('run_live_trade.get_open_trades') as mock_get_trades, \
     patch('run_live_trade.place_trade') as mock_place_trade:

    # Configure mocks
    mock_get_client.return_value = MagicMock()
    mock_load_model.return_value = (MagicMock(), ['feature1'])
    mock_get_details.return_value = (-4, 5)
    mock_get_candles.return_value = sample_df_with_indicators
    mock_get_decision.return_value = ("BUY", "1.10000", "1.12000")
    mock_get_trades.return_value = None
    
    main()

    print("--- Verifying Scenario 1 Results ---")
    assert mock_get_decision.call_count == len(INSTRUMENTS)
    print(f"✅ get_trade_decision was called for all {len(INSTRUMENTS)} instruments.")
    assert mock_place_trade.call_count == len(INSTRUMENTS)
    print(f"✅ place_trade was correctly called {mock_place_trade.call_count} time(s).")


# --- SCENARIO 2: A trade signal is found BUT an open trade ALREADY exists ---
print("\n--- SCENARIO 2: Testing when a trade signal is found but a trade is already open ---")

with patch('run_live_trade.get_oanda_client') as mock_get_client, \
     patch('run_live_trade.load_model_and_features') as mock_load_model, \
     patch('run_live_trade.get_instrument_details') as mock_get_details, \
     patch('run_live_trade.get_historical_candles') as mock_get_candles, \
     patch('run_live_trade.get_trade_decision') as mock_get_decision, \
     patch('run_live_trade.get_open_trades') as mock_get_trades, \
     patch('run_live_trade.place_trade') as mock_place_trade:

    # --- Configure mocks (FIXED) ---
    mock_get_client.return_value = MagicMock()
    mock_load_model.return_value = (MagicMock(), ['feature1']) # <-- This was missing
    mock_get_details.return_value = (-4, 5)                   # <-- This was missing
    mock_get_candles.return_value = sample_df_with_indicators
    mock_get_decision.return_value = ("SELL", "1.20000", "1.18000")
    mock_get_trades.return_value = {'id': '123', 'instrument': 'EUR_USD'}
    
    main()

    print("--- Verifying Scenario 2 Results ---")
    assert mock_place_trade.call_count == 0, f"Expected place_trade to be called 0 times, but was {mock_place_trade.call_count}"
    print("✅ place_trade was correctly NOT called because an open trade exists.")


# --- SCENARIO 3: NO trade signal is found ---
print("\n--- SCENARIO 3: Testing when NO trade signal is found ---")

with patch('run_live_trade.get_oanda_client') as mock_get_client, \
     patch('run_live_trade.load_model_and_features') as mock_load_model, \
     patch('run_live_trade.get_instrument_details') as mock_get_details, \
     patch('run_live_trade.get_historical_candles') as mock_get_candles, \
     patch('run_live_trade.get_trade_decision') as mock_get_decision, \
     patch('run_live_trade.place_trade') as mock_place_trade:

    # --- Configure mocks (FIXED) ---
    mock_get_client.return_value = MagicMock()
    mock_load_model.return_value = (MagicMock(), ['feature1']) # <-- This was missing
    mock_get_details.return_value = (-4, 5)                   # <-- This was missing
    mock_get_candles.return_value = sample_df_with_indicators
    mock_get_decision.return_value = (None, None, None)
    
    main()

    print("--- Verifying Scenario 3 Results ---")
    assert mock_place_trade.call_count == 0, f"Expected place_trade to be called 0 times, but was {mock_place_trade.call_count}"
    print("✅ place_trade was correctly NOT called because no signal was generated.")

print("\n--- All Tests Complete ---")

--- Setting up test environment for run_live_trade.py ---

--- SCENARIO 1: Testing a successful trade signal ---
Starting trading bot...

=== Checking EUR_USD ===
Signal: BUY EUR_USD 1000 units SL=1.10000 TP=1.12000
Trade execution may have failed or was rejected.

=== Checking USD_JPY ===
Signal: BUY USD_JPY 1000 units SL=1.10000 TP=1.12000
Trade execution may have failed or was rejected.

=== Checking GBP_USD ===
Signal: BUY GBP_USD 1000 units SL=1.10000 TP=1.12000
Trade execution may have failed or was rejected.

=== Checking USD_CHF ===
Signal: BUY USD_CHF 1000 units SL=1.10000 TP=1.12000
Trade execution may have failed or was rejected.

=== Checking AUD_USD ===
Signal: BUY AUD_USD 1000 units SL=1.10000 TP=1.12000
Trade execution may have failed or was rejected.

=== Checking USD_CAD ===
Signal: BUY USD_CAD 1000 units SL=1.10000 TP=1.12000
Trade execution may have failed or was rejected.

=== Checking NZD_USD ===
Signal: BUY NZD_USD 1000 units SL=1.10000 TP=1.12000
Trade execution 

In [ ]:
# Test cell for the email notification service

import os
from unittest.mock import patch, MagicMock

# Import the function to be tested and config variables
from run_live_trade import send_email_notification
from live_trader_config import (
    EMAIL_NOTIFICATIONS_ENABLED, 
    EMAIL_SENDER, 
    EMAIL_RECIPIENT
)

# --- PART 1: Live End-to-End Test ---
# This will attempt to send a REAL email using your configuration.
#
# PRE-REQUISITES:
# 1. Ensure your `live_trader_config.py` has the correct sender/recipient emails.
# 2. Ensure you have set the `EMAIL_APP_PASSWORD` environment variable in the terminal
#    from which you launched Jupyter Notebook.
# 3. If it fails, check your email for any "Critical security alert" from Google.
#    You may need to confirm the sign-in attempt was you.

print("--- 1. Attempting to send a live test email... ---")

# Check if the password environment variable is set before trying
if os.getenv("EMAIL_APP_PASSWORD"):
    print(f"Sending test email from '{EMAIL_SENDER}' to '{EMAIL_RECIPIENT}'.")
    try:
        send_email_notification(
            subject="Trading Bot Test Email",
            body="This is a test notification from your trading bot script. If you received this, the email configuration is working."
        )
        print("✅ Live email test function executed. Check your inbox.")
    except Exception as e:
        print(f"❌ An unexpected error occurred during the live test: {e}")
else:
    print("❌ SKIPPED: The 'EMAIL_APP_PASSWORD' environment variable is not set.")
    print("   Please set it in your terminal and restart Jupyter Notebook to run this test.")


# --- PART 2: Automated Logic Tests (using Mocking) ---
# These tests verify the function's logic without sending real emails.

print("\n--- 2. Running automated logic tests... ---")

# Test Case A: Verify it does nothing if notifications are disabled
@patch('run_live_trade.smtplib.SMTP_SSL')
def test_notifications_disabled(mock_smtp):
    print("\n--- Testing with notifications disabled ---")
    with patch('run_live_trade.EMAIL_NOTIFICATIONS_ENABLED', False):
        send_email_notification("Test Subject", "Test Body")
        # Assert that the SMTP library was never called
        assert not mock_smtp.called, "SMTP was called even when notifications were disabled."
        print("✅ Correctly did nothing when notifications are disabled.")

# Test Case B: Verify it fails gracefully if the password is not set
@patch('run_live_trade.smtplib.SMTP_SSL')
def test_missing_password(mock_smtp):
    print("\n--- Testing with missing email password ---")
    # Temporarily remove the environment variable for this test
    with patch.dict(os.environ, {}, clear=True):
        send_email_notification("Test Subject", "Test Body")
        # Assert that the SMTP library was never called
        assert not mock_smtp.called, "SMTP was called even when the password was missing."
        print("✅ Correctly did nothing when the password was not set.")

# Run the automated tests
test_notifications_disabled()
test_missing_password()

print("\n--- All Tests Complete ---")

--- 1. Attempting to send a live test email... ---
Sending test email from 'vidulmetallica@gmail.com' to 'vidulmetallica@gmail.com'.
Email notification sent successfully.
✅ Live email test function executed. Check your inbox.

--- 2. Running automated logic tests... ---

--- Testing with notifications disabled ---
✅ Correctly did nothing when notifications are disabled.

--- Testing with missing email password ---
Email notification failed: Missing sender, recipient, or EMAIL_APP_PASSWORD environment variable.
✅ Correctly did nothing when the password was not set.

--- All Tests Complete ---
